In [ ]:
import os
from pathlib import Path

# Vérifier que les fichiers sont bien là (optionnel mais recommandé)
dataset_dir = Path('/kaggle/input/augmented-savi-640/Dataset_B_640x640')
working_dir = Path('/kaggle/working/')
print("Contenu du dossier :")
!ls {dataset_dir}

In [ ]:
# --- Création du Fichier YAML ---

# Contenu du fichier de configuration.
# Le 'path' doit pointer vers le dossier racine du dataset.
# Les chemins 'train', 'val', 'test' sont relatifs à ce 'path'.
yaml_content = f"""
path: {dataset_dir.as_posix()}
train: images/train
val: images/val
test: images/test

names:
  0: Person
  1: Bicycle
  2: Car
  3: Cattle
"""

# Écriture du contenu dans un fichier .yaml dans le répertoire de travail
yaml_file_path = working_dir / 'dataset.yaml'
with open(yaml_file_path, 'w') as f:
    f.write(yaml_content)

print(f"Fichier de configuration créé avec succès à l'emplacement : {yaml_file_path}")
print("\n--- Contenu du YAML ---")
!cat {yaml_file_path}

In [ ]:
import pandas as pd

# Chargez votre fichier CSV.
metadata_path = dataset_dir / 'metadata_640x640.csv'
df = pd.read_csv(metadata_path)

print("--- 5 premières lignes du DataFrame ---")
display(df.head())

print("\n--- Informations générales sur le DataFrame ---")
df.info()

print("\n--- Statistiques descriptives des colonnes numériques ---")
display(df.describe())

print("\n--- Valeurs uniques dans les colonnes catégorielles ---")
print(f"Meteo: {df['meteo'].unique()}")
print(f"Region: {df['region'].unique()}")
print(f"Mode: {df['mode'].unique()}")

In [ ]:
# 1) si l'id contient "Tankpe" -> region = "urban periphery"
df.loc[df['id'].str.contains('Tankpe', case=False, na=False), 'region'] = 'urban periphery'

# 2) si l'id contient "Godomey" -> region = "urban"
df.loc[df['id'].str.contains('Godomey', case=False, na=False), 'region'] = 'urban'

# 3) normaliser la colonne meteo : "Sunny" -> "sunny" et "Night" -> "night"
# méthode robuste : enlever espaces puis tout mettre en minuscules
df['meteo'] = df['meteo'].astype(str).str.strip().str.lower()

# vérifications rapides
print("Valeurs uniques dans 'region' après modifs :", df['region'].unique())
print("Valeurs uniques dans 'meteo' après modifs  :", df['meteo'].unique())

# (optionnel) afficher quelques lignes concernées pour contrôle
print("\nExemples d'entrées contenant 'Tankpe' :")
print(df[df['id'].str.contains('Tankpe', case=False, na=False)].head())

print("\nExemples d'entrées contenant 'Godomey' :")
print(df[df['id'].str.contains('Godomey', case=False, na=False)].head())


In [ ]:
# Sélection des colonnes catégorielles à encoder
categorical_cols = ['meteo', 'region', 'mode']

# Application de l'encodage one-hot
df_encoded = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols)

print("--- DataFrame après encodage one-hot ---")
display(df_encoded.head())

# Garder en mémoire les colonnes créées pour pouvoir les réutiliser à l'inférence
encoded_cols = [col for col in df_encoded.columns if any(cat_col in col for cat_col in categorical_cols)]
print(f"\nColonnes créées par l'encodage : {encoded_cols}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pickle

# Sélection des colonnes numériques à normaliser
numerical_cols = ['angle', 'altitude', 'y_start', 'y_end']

# Initialisation du scaler Min-Max
scaler = MinMaxScaler()

# Application du scaler sur nos données
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

print("--- DataFrame après normalisation des données numériques ---")
display(df_encoded.head())

# --- CRUCIAL : Sauvegarde du scaler ---
# Nous en aurons besoin plus tard pour transformer les données de validation/test
# avec EXACTEMENT la même échelle apprise sur les données d'entraînement.
scaler_path = '/kaggle/working/min_max_scaler.pkl'
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

print(f"\nScaler sauvegardé à l'emplacement : {scaler_path}")

In [ ]:
# Mettre la colonne 'id' comme index pour une recherche facile plus tard
df_processed = df_encoded.set_index('id')

print("--- DataFrame final prêt pour l'entraînement ---")
display(df_processed.head())

print("\n--- Dimensions du vecteur de caractéristiques pour le MLP ---")
print(f"Chaque image sera représentée par un vecteur de {df_processed.shape[1]} features.")

# Sauvegarder le DataFrame traité pour une utilisation future
processed_data_path = '/kaggle/working/processed_metadata.csv'
df_processed.to_csv(processed_data_path)

print(f"\nDonnées traitées sauvegardées à l'emplacement : {processed_data_path}")

In [ ]:
# --- Installation ---
# On installe la bibliothèque ultralytics qui contient l'implémentation de YOLOv8.
# Le flag '-q' (quiet) permet de réduire la quantité de logs durant l'installation.
!pip install ultralytics -q

print("Installation terminée.")

In [ ]:
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np

class MultimodalDataset(Dataset):
    """
    Dataset PyTorch personnalisé pour charger des images, leurs labels YOLO,
    et des métadonnées tabulaires associées.
    """
    def __init__(self, images_dir, labels_dir, metadata_df):
        """
        Args:
            images_dir (str): Chemin vers le dossier contenant les images.
            labels_dir (str): Chemin vers le dossier contenant les fichiers de labels (.txt).
            metadata_df (pd.DataFrame): DataFrame contenant les métadonnées prétraitées.
                                        L'index du DataFrame doit être l'ID de l'image.
        """
        self.images_dir = Path(images_dir)
        self.labels_dir = Path(labels_dir)
        self.metadata_df = metadata_df
        
        # Obtenir tous les noms de fichiers image (sans extension)
        all_image_stems = {p.stem for p in self.images_dir.glob('*.jpg')}
        
        # Filtrer pour ne garder que les IDs qui ont une entrée dans le metadata_df
        self.image_ids = sorted([
            stem for stem in all_image_stems
            if stem in self.metadata_df.index
        ])
        
        # Avertissement si des images n'ont pas de métadonnées
        if len(all_image_stems) != len(self.image_ids):
            missing_count = len(all_image_stems) - len(self.image_ids)
            print(f"Attention : {missing_count} images dans {images_dir} n'ont pas de métadonnées correspondantes et seront ignorées.")


    def __len__(self):
        """Retourne le nombre total d'échantillons dans le dataset."""
        return len(self.image_ids)

    def __getitem__(self, idx):
        """
        Récupère un échantillon (image, labels, métadonnées) à l'index donné.
        """
        # 1. Obtenir l'ID de l'image
        image_id = self.image_ids[idx]
        
        # 2. Charger l'image
        image_path = self.images_dir / f"{image_id}.jpg"
        image = cv2.imread(str(image_path))
        target_size = (640, 640)
        if image.shape[:2] != (target_size[1], target_size[0]):
             image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_tensor = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
        
        # 3. Charger les labels
        labels = []
        label_path = self.labels_dir / f"{image_id}.txt"
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    labels.append([float(p) for p in parts])
        labels_tensor = torch.tensor(labels, dtype=torch.float32)
        
        # 4. Récupérer les métadonnées
        metadata_vector = self.metadata_df.loc[image_id].values.astype(np.float32)
        metadata_tensor = torch.from_numpy(metadata_vector)
        
        # 5. Retourner un dictionnaire
        return {
            'image': image_tensor,
            'labels': labels_tensor,
            'metadata': metadata_tensor,
            'id': image_id
        }

print("Classe MultimodalDataset définie avec succès.")

In [ ]:
# --- Configuration des Chemins ---
BASE_DATA_DIR = Path('/kaggle/input/augmented-savi-640/Dataset_B_640x640') # D'après votre notebook
METADATA_PATH = '/kaggle/working/processed_metadata.csv' # Le fichier que nous avons créé à l'étape 1

# Définir les chemins spécifiques pour chaque sous-ensemble
images_train_dir = BASE_DATA_DIR / 'images' / 'train'
labels_train_dir = BASE_DATA_DIR / 'labels' / 'train'

images_val_dir = BASE_DATA_DIR / 'images' / 'val'
labels_val_dir = BASE_DATA_DIR / 'labels' / 'val'

images_test_dir = BASE_DATA_DIR / 'images' / 'test'
labels_test_dir = BASE_DATA_DIR / 'labels' / 'test'

# --- Chargement des Métadonnées ---
df_processed = pd.read_csv(METADATA_PATH, index_col='id')
print(f"Métadonnées chargées avec {len(df_processed)} entrées.")

# --- Instanciation des Datasets ---
print("\nInstanciation des datasets...")

train_dataset = MultimodalDataset(
    images_dir=images_train_dir,
    labels_dir=labels_train_dir,
    metadata_df=df_processed
)

val_dataset = MultimodalDataset(
    images_dir=images_val_dir,
    labels_dir=labels_val_dir,
    metadata_df=df_processed
)

test_dataset = MultimodalDataset(
    images_dir=images_test_dir,
    labels_dir=labels_test_dir,
    metadata_df=df_processed
)

# --- Vérification ---
print("\n--- Vérification des tailles des datasets ---")
print(f"Nombre d'échantillons dans le set d'entraînement : {len(train_dataset)}")
print(f"Nombre d'échantillons dans le set de validation   : {len(val_dataset)}")
print(f"Nombre d'échantillons dans le set de test         : {len(test_dataset)}")

# --- Test sur un échantillon du set de validation ---
if len(val_dataset) > 0:
    print("\n--- Test sur le premier échantillon du set de validation ---")
    
    sample = val_dataset[0]
    
    print(f"ID de l'image : {sample['id']}")
    print(f"Clés retournées : {list(sample.keys())}")
    
    img_tensor = sample['image']
    lbl_tensor = sample['labels']
    meta_tensor = sample['metadata']
    
    print(f"Image - Shape: {img_tensor.shape}, Type: {img_tensor.dtype}")
    print(f"Labels - Shape: {lbl_tensor.shape}, Type: {lbl_tensor.dtype}")
    print(f"Metadata - Shape: {meta_tensor.shape}, Type: {meta_tensor.dtype}")
    
    # Vérifiez que le nombre de features des métadonnées correspond bien
    expected_features = df_processed.shape[1]
    print(f"Le vecteur de métadonnées a {meta_tensor.shape[0]} features (attendu: {expected_features}).")

else:
    print("\nAttention : Le dataset de validation est vide. Veuillez vérifier les chemins d'accès.")

In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    """
    Un Multi-Layer Perceptron simple pour traiter les métadonnées tabulaires.
    """
    def __init__(self, input_size, output_size=512):
        """
        Args:
            input_size (int): La taille du vecteur de métadonnées d'entrée.
            output_size (int): La taille du vecteur de caractéristiques en sortie (embedding).
        """
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.1), # Ajout de dropout pour la régularisation
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, output_size)
        )

    def forward(self, x):
        """Passe avant du MLP."""
        return self.layers(x)

# --- Test rapide du MLP ---
# Récupérer la taille d'entrée depuis nos données prétraitées
metadata_df = pd.read_csv('/kaggle/working/processed_metadata.csv')
input_features = metadata_df.shape[1] - 1 # -1 car la colonne 'id' est l'index

# Instancier le MLP
mlp_model = MLP(input_size=input_features)

# Créer un faux tenseur de métadonnées (batch de 4)
dummy_metadata = torch.randn(4, input_features)

# Faire une passe avant
output_embedding = mlp_model(dummy_metadata)

print(f"--- Test du MLP ---")
print(f"Taille du vecteur d'entrée : {input_features}")
print(f"Shape de l'entrée du MLP : {dummy_metadata.shape}")
print(f"Shape de la sortie (embedding) du MLP : {output_embedding.shape}") # Devrait être [4, 512]

In [ ]:
import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    """Channel-attention module https://github.com/open-mmlab/mmdetection/tree/v3.0.0rc1/configs/rtmdet."""

    def __init__(self, channels: int) -> None:
        """Initializes the class and sets the basic configurations and instance variables required."""
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Conv2d(channels, channels, 1, 1, 0, bias=True)
        self.act = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Applies forward pass using activation on convolutions of the input, optionally using batch normalization."""
        return x * self.act(self.fc(self.pool(x)))


class SpatialAttention(nn.Module):
    """Spatial-attention module."""

    def __init__(self, kernel_size=7):
        """Initialize Spatial-attention module with kernel size argument."""
        super().__init__()
        assert kernel_size in {3, 7}, "kernel size must be 3 or 7"
        padding = 3 if kernel_size == 7 else 1
        self.cv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.act = nn.Sigmoid()

    def forward(self, x):
        """Apply channel and spatial attention on input for feature recalibration."""
        return x * self.act(self.cv1(torch.cat([torch.mean(x, 1, keepdim=True), torch.max(x, 1, keepdim=True)[0]], 1)))


class CBAM(nn.Module):
    """Convolutional Block Attention Module."""

    def __init__(self, c1, kernel_size=7):
        """Initialize CBAM with given input channel (c1) and kernel size."""
        super().__init__()
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        """Applies the forward pass through C1 module."""
        return self.spatial_attention(self.channel_attention(x))

print("Module CBAM définis avec succès.")

In [ ]:
# --- Étape 1 : Importer le parseur de modèles ---
from ultralytics.nn import tasks

# --- Enregistrer notre module personnalisé ---
tasks.CBAM = CBAM
print("Module CBAM enregistré avec succès.")

# --- Création du Fichier de Configuration YAML Final ---

yaml_config_content = """
# Ultralytics YOLO 🚀, AGPL-3.0 license
# Fichier de configuration pour YOLOv8s avec des blocs C2f_CBAM

# Paramètres
nc: 4 
scales:
  # [depth, width, max_channels]
  s: [0.33, 0.50, 1024]  #

backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]  # 9

head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 10
  - [-1, 1, CBAM, [512]]  # Add CBAM after Upsample
  - [[-1, 6], 1, Concat, [1]]  # 12 cat backbone P4
  - [-1, 3, C2f, [512, False]]  # 13

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 14
  - [-1, 1, CBAM, [256]]  # Add CBAM after Upsample
  - [[-1, 4], 1, Concat, [1]]  # 16 cat backbone P3
  - [-1, 3, C2f, [256, False]]  # 17

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 18
  - [-1, 1, CBAM, [128]]  # Add CBAM after Upsample
  - [[-1, 2], 1, Concat, [1]]  # 20 cat backbone P2
  - [-1, 1, C2f, [128, False]]  # 21

  - [-1, 1, Conv, [128, 3, 2]]  # 22
  - [[-1, 17], 1, Concat, [1]]  # 23 cat head P3
  - [-1, 3, C2f, [256, False]]  # 24

  - [-1, 1, Conv, [256, 3, 2]]  # 25
  - [[-1, 13], 1, Concat, [1]]  # 26 cat head P4
  - [-1, 3, C2f, [512, False]]  # 27

  - [-1, 1, Conv, [512, 3, 2]]  # 28
  - [[-1, 9], 1, Concat, [1]]  # 29 cat head P5
  - [-1, 3, C2f, [1024, False]]  # 30

  - [[21, 24, 27, 30], 1, Detect, [nc]]  # 31 Detect(P2, P3, P4, P5)
"""

# Écrire ce contenu dans un fichier .yaml dans le répertoire de travail
custom_yaml_path = working_dir / 'yolov8s-cbam.yaml'
with open(custom_yaml_path, 'w') as f:
    f.write(yaml_config_content)

print(f"Fichier de configuration YAML personnalisé créé : {custom_yaml_path}")

In [ ]:
from ultralytics.nn.tasks import DetectionModel
from ultralytics.nn.modules import Concat, C2f, Conv

class YOLOv8Multimodal(nn.Module):
    """
    Modèle multimodal qui fusionne les caractéristiques d'un backbone YOLOv8
    avec des métadonnées via un MLP. (Version corrigée)
    """
    def __init__(self, yolo_cfg_path, metadata_input_size, num_classes):
        super().__init__()
        
        # 1. Charger le modèle YOLO de base.
        self.yolo_model = DetectionModel(cfg=yolo_cfg_path, nc=num_classes)

        self.model = self.yolo_model.model
        
        # 2. Isoler la tête de détection. C'est notre source de vérité.
        self.detect_head = self.model[-1]
        
        # 3. Instancier notre MLP
        metadata_embedding_size = 512
        self.metadata_mlp = MLP(input_size=metadata_input_size, output_size=metadata_embedding_size)
        
        # 4. --- SOLUTION CORRIGÉE : Accès correct aux propriétés des couches ---
        self.fusion_indices = [21, 24, 27, 30]
        self.fusion_convs = nn.ModuleList()
        
        print("Détermination dynamique des canaux en inspectant la tête 'Detect'...")
        
        # self.detect_head.nl est le nombre de couches de détection (4 dans notre cas)
        for i in range(self.detect_head.nl):
            # CORRECTION : Accéder correctement aux propriétés de la convolution
            # La classe Conv d'Ultralytics a un attribut 'conv' qui contient la vraie Conv2d de PyTorch
            try:
                # Méthode 1 : Essayer d'accéder via l'attribut conv
                if hasattr(self.detect_head.cv2[i][0], 'conv'):
                    image_channels = self.detect_head.cv2[i][0].conv.in_channels
                # Méthode 2 : Essayer d'accéder directement si c'est déjà une Conv2d
                elif hasattr(self.detect_head.cv2[i][0], 'in_channels'):
                    image_channels = self.detect_head.cv2[i][0].in_channels
                # Méthode 3 : Inspection des paramètres du module
                else:
                    # Récupérer les paramètres du premier module Conv
                    conv_module = self.detect_head.cv2[i][0]
                    # Les modules Conv d'Ultralytics stockent leurs paramètres différemment
                    for name, param in conv_module.named_parameters():
                        if 'weight' in name:
                            image_channels = param.shape[1]  # in_channels est la 2ème dimension
                            break
                    else:
                        # Fallback : utiliser une valeur par défaut basée sur l'index
                        default_channels = [64, 128, 256, 512]
                        image_channels = default_channels[i] if i < len(default_channels) else 512
                        print(f"  - Attention: Utilisation de la valeur par défaut pour la branche {i}: {image_channels} canaux")
                        
            except Exception as e:
                # En cas d'erreur, utiliser des valeurs par défaut raisonnables
                default_channels = [64, 128, 256, 512]
                image_channels = default_channels[i] if i < len(default_channels) else 512
                print(f"  - Erreur lors de l'inspection de la branche {i}: {e}")
                print(f"  - Utilisation de la valeur par défaut: {image_channels} canaux")
            
            print(f"  - Branche {i} (entrée de la couche {self.fusion_indices[i]}): {image_channels} canaux d'image requis.")
            
            # Créer la couche de fusion correspondante avec les bonnes dimensions
            fusion_layer = self._create_fusion_layer(image_channels, metadata_embedding_size)
            self.fusion_convs.append(fusion_layer)

    def _create_fusion_layer(self, image_channels, metadata_channels):
        """Crée une petite couche de convolution pour réduire la dimension après la fusion."""
        return nn.Sequential(
            nn.Conv2d(image_channels + metadata_channels, image_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(image_channels),
            nn.SiLU()
        )

    def forward(self, image, metadata):
        """
        La passe avant du modèle multimodal.
        """
        metadata_embedding = self.metadata_mlp(metadata)

        y = []
        fusion_sources = {}
        for i, module in enumerate(self.model[:-1]):
            if module.f == -1:
                x = y[-1] if y else image
            else:
                x = [y[j] for j in module.f]
            
            x = module(x)
            y.append(x)
            
            if i in self.fusion_indices:
                fusion_sources[i] = x
        
        yolo_outputs = [fusion_sources[i] for i in self.fusion_indices]
        fused_features = []

        for yolo_out, fusion_conv in zip(yolo_outputs, self.fusion_convs):
            b, c, h, w = yolo_out.shape
            meta_emb = metadata_embedding.unsqueeze(-1).unsqueeze(-1).expand(b, -1, h, w)
            fused_out = torch.cat([yolo_out, meta_emb], dim=1)
            fused_features.append(fusion_conv(fused_out))
        
        return self.detect_head(fused_features)

print("Classe YOLOv8Multimodal (version corrigée) définie avec succès.")

In [ ]:
# --- Paramètres de configuration ---
YOLO_CFG_PATH = '/kaggle/working/yolov8s-cbam.yaml' # Le YAML que vous avez créé
METADATA_INPUT_SIZE = input_features # Calculé dans la cellule 3.1
NUM_CLASSES = 4 # Person, Bicycle, Car, Cattle

# --- Instanciation du modèle complet ---
try:
    multimodal_model = YOLOv8Multimodal(
        yolo_cfg_path=YOLO_CFG_PATH,
        metadata_input_size=METADATA_INPUT_SIZE,
        num_classes=NUM_CLASSES
    )
    print("Modèle multimodal instancié avec succès.")
    
    # --- Création de données d'entrée factices ---
    BATCH_SIZE = 2
    IMG_SIZE = 640
    dummy_images = torch.randn(BATCH_SIZE, 3, IMG_SIZE, IMG_SIZE)
    dummy_metadata = torch.randn(BATCH_SIZE, METADATA_INPUT_SIZE)
    
    # Mettre le modèle en mode évaluation pour le test
    multimodal_model.eval()
    
    # --- Passe avant ---
    with torch.no_grad():
        print("\nExécution d'une passe avant (dry run)...")
        predictions = multimodal_model(dummy_images, dummy_metadata)
    
    print("Passe avant réussie !")
    
    # --- Analyse de la sortie ---
    # La sortie de la tête de détection de YOLOv8 est une liste de tenseurs
    # (un pour chaque échelle de prédiction).
    print(f"\nType de la sortie : {type(predictions)}")
    print(f"Nombre de tenseurs en sortie : {len(predictions)}")
    
    # Le premier tenseur contient les prédictions (boîtes, scores de classe, score de confiance)
    # Sa shape est [batch_size, num_classes + 4 (pour la boîte), num_predictions]
    print(f"Shape du premier tenseur de prédiction : {predictions[0].shape}")
    
except Exception as e:
    print(f"\nUne erreur est survenue lors de l'instanciation ou du test du modèle : {e}")
    import traceback
    traceback.print_exc()

In [ ]:
def freeze_yolo_backbone(model):
    """Gèle tous les poids du backbone yolo_model."""
    print("Gel des poids du backbone YOLO...")
    for name, param in model.named_parameters():
        if 'yolo_model' in name:
            param.requires_grad = False

def unfreeze_yolo_backbone(model):
    """Dégèle tous les poids du backbone yolo_model."""
    print("Dégel des poids du backbone YOLO...")
    for name, param in model.named_parameters():
        if 'yolo_model' in name:
            param.requires_grad = True

def check_frozen_status(model):
    """Vérifie et affiche le statut (gelé/dégelé) des différents groupes de paramètres."""
    print("\n--- Statut des Paramètres ---")
    status = {"yolo_model": True, "metadata_mlp": False, "fusion_convs": False}
    for name, param in model.named_parameters():
        group = name.split('.')[0]
        if group not in status:
            status[group] = param.requires_grad
        else:
            status[group] = status[group] and param.requires_grad
    
    for group, is_trainable in status.items():
        print(f"  - Groupe '{group}': {'Entraînable' if is_trainable else 'Gelé'}")
    print("----------------------------\n")

print("Fonctions de gel/dégel définies.")

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from pathlib import Path
import yaml
import copy

# Importer directement la classe de la fonction de perte
from ultralytics.utils.loss import v8DetectionLoss

# --- 1. Hyperparamètres et Configuration ---
EPOCHS = 300
BATCH_SIZE = 8
LEARNING_RATE = 1e-3
PROJECT_NAME = 'multimodal_runs_pure' # Nouveau nom pour ne pas tout mélanger
EXPERIMENT_NAME = 'exp_final'

# Créer le répertoire de sauvegarde
save_dir = Path(f'/kaggle/working/{PROJECT_NAME}/{EXPERIMENT_NAME}')
save_dir.mkdir(parents=True, exist_ok=True)
weights_dir = save_dir / 'weights'
weights_dir.mkdir(exist_ok=True)

# --- 2. Modèle, Optimiseur, Scheduler ---
# (On suppose que les DataLoaders train_loader et val_loader existent déjà)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation du device : {device}")

multimodal_model.to(device)

# Geler le backbone pour la Phase 1
freeze_yolo_backbone(multimodal_model)
check_frozen_status(multimodal_model)

# L'optimiseur ne voit que les paramètres entraînables ---
# C'est la méthode standard pour un entraînement avec des couches gelées.
trainable_params = filter(lambda p: p.requires_grad, multimodal_model.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# --- 3. Instanciation de la Fonction de Perte ---
# On a besoin d'un objet 'args' factice pour la fonction de perte
from types import SimpleNamespace
# Ces valeurs sont les poids par défaut de la perte dans ultralytics
args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5) 
multimodal_model.args = args

# La perte a aussi besoin de connaître la tête de détection
multimodal_model.model = multimodal_model.yolo_model.model

loss_fn = v8DetectionLoss(multimodal_model)

print("Configuration pure terminée. Prêt pour l'entraînement.")

In [ ]:

def validate_model(model, loader, loss_function, device):
    """
    Fonction de validation simple qui calcule la perte moyenne sur l'ensemble de validation.
    """
    model.eval()  # Passer le modèle en mode évaluation
    total_val_loss = 0.0
    pbar_val = tqdm(loader, desc="[Validation]")

    with torch.no_grad():  # Pas de calcul de gradient pendant la validation
        for batch in pbar_val:
            images = batch['image'].to(device)
            metadata = batch['metadata'].to(device)
            targets = batch['labels'].to(device)
            
            # Gérer le cas où un batch de validation n'a aucune cible
            if targets.numel() == 0:
                continue

            preds = model(images, metadata)
            
            batch_for_loss = {
                'imgs': images,
                'batch_idx': targets[:, 0],
                'cls': targets[:, 1],
                'bboxes': targets[:, 2:]
            }

            loss, loss_items = loss_function(preds, batch_for_loss)
            total_val_loss += loss.sum().item()
            
            pbar_val.set_postfix(val_loss=f'{total_val_loss / (pbar_val.n + 1):.4f}')
            
    return total_val_loss / len(loader)

print("Fonction de validation définie.")

In [ ]:
best_val_loss = float('inf')
NUM_EPOCHS_FREEZE = 100 # Le nombre d'époques pour la Phase 1

# Boucle principale sur les époques
for epoch in range(EPOCHS):
    if epoch == NUM_EPOCHS_FREEZE:
        unfreeze_yolo_backbone(multimodal_model)
        check_frozen_status(multimodal_model)
        
        print("Phase 2 : Dégel et création d'un nouvel optimiseur avec un learning rate plus faible.")
        # On entraîne maintenant TOUS les paramètres avec un LR plus faible
        optimizer = torch.optim.AdamW(multimodal_model.parameters(), lr=LEARNING_RATE / 10)
        # On peut optionnellement réinitialiser le scheduler
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS - NUM_EPOCHS_FREEZE)
    multimodal_model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Training]")
    total_train_loss = 0.0
    
    for i, batch in enumerate(pbar):
        images = batch['image'].to(device)
        metadata = batch['metadata'].to(device)
        targets = batch['labels'].to(device)
        
        # Sauter les batchs sans aucune annotation
        if targets.numel() == 0:
            continue
            
        optimizer.zero_grad()
        
        preds = multimodal_model(images, metadata)
        
        batch_for_loss = {
            'imgs': images,
            'batch_idx': targets[:, 0],
            'cls': targets[:, 1],
            'bboxes': targets[:, 2:]
        }

        loss, loss_items = loss_fn(preds, batch_for_loss)
        
        # --- DEBUG : Afficher les composantes de la perte pour le premier batch ---
        if i == 0:
            print(f"\nComposantes de la perte (1er batch): {loss_items}")
            
        loss_scalar = loss.sum()
        loss_scalar.backward()
        optimizer.step()
        
        total_train_loss += loss_scalar.item()
        pbar.set_postfix(train_loss=f'{total_train_loss / (i + 1):.4f}')
        
    scheduler.step()

    # --- Validation à la fin de chaque époque ---
    avg_val_loss = validate_model(multimodal_model, val_loader, loss_fn, device)
    print(f"\nEpoch {epoch+1} - Perte d'entraînement moyenne: {total_train_loss / len(train_loader):.4f} - Perte de validation moyenne: {avg_val_loss:.4f}")

    # --- Sauvegarde des modèles ---
    model_to_save = multimodal_model.module if hasattr(multimodal_model, 'module') else multimodal_model
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model_to_save.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_loss': avg_val_loss
    }

    # Sauvegarder le dernier modèle
    torch.save(checkpoint, weights_dir / 'last.pt')

    # Sauvegarder le meilleur modèle (basé sur la perte de validation)
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(checkpoint, weights_dir / 'best.pt')
        print(f"  -> Nouveau meilleur modèle sauvegardé avec une perte de validation de : {avg_val_loss:.4f}")
        
print("\n--- Entraînement terminé ! ---")